In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Training and deploying a tabular model using Vertex AutoML.

![Training pipeline](../images/automl.png)

## Install required packages

In [ ]:
# Get the site-packages directory so we can remove invalid packages.
import site
sp = site.getsitepackages()[0]
print(sp)

In [ ]:
%%bash -s "$sp"
# Remove the invalide site-packages
echo $1
sudo rm -rf $1/~*

In [ ]:
%%bash
pip install --user google-cloud-aiplatform
pip install --user kfp
pip install --user google-cloud-pipeline-components
pip install --user google-cloud-bigquery-datatransfer


### Restart the kernel
Once you've installed the required packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Import the required packages

In [ ]:
import os
import pprint
import pandas as pd
import tensorflow as tf
import time
from datetime import datetime
import csv

import matplotlib.pyplot as plt

import google.auth

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1beta1 import types
from google.cloud import bigquery
from google.cloud import exceptions

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow_io import bigquery as tfio_bq


## Configure GCP settings

*Before running the notebook make sure to follow the repo's README file to install the pre-requisites and configure GCP authentication.*

In [ ]:
creds, PROJECT = google.auth.default()
print(creds)
REGION = 'us-central1'

STAGING_BUCKET = f'gs://{PROJECT}-labs'

# Get the configured service account this notebook is running as
bash_output = !gcloud config list account --format "value(core.account)" 2> /dev/null
VERTEX_SA = bash_output[0]

print(f"PROJECT = {PROJECT}")
print(f"STAGING_BUCKET = {STAGING_BUCKET}")
print(f"VERTEX_SA = {VERTEX_SA}")

# Create the bucket. Ignore error if it already exists.
!gsutil mb -l $REGION $STAGING_BUCKET

## Preparing training data in BigQuery

### Explore Chicago Taxi dataset

In [ ]:
%%bigquery data

SELECT 
    *
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
LIMIT 3

In [ ]:
data.head().T

In [ ]:
%%bigquery data

SELECT 
    CAST(EXTRACT(DAYOFWEEK FROM trip_start_timestamp) AS string) AS trip_dayofweek, 
    FORMAT_DATE('%A',cast(trip_start_timestamp as date)) AS trip_dayname,
    COUNT(*) as trip_count,
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    EXTRACT(YEAR FROM trip_start_timestamp) = 2020 
GROUP BY
    trip_dayofweek,
    trip_dayname
ORDER BY
    trip_dayofweek

In [ ]:
data

In [ ]:
data.plot(kind='bar', x='trip_dayname', y='trip_count')

### Create data splits

In [ ]:
BQ_DATASET_NAME = f'vertex_lab01' 
BQ_TABLE_NAME = 'features'
BQ_LOCATION = 'US'
SAMPLE_SIZE = 500000
YEAR = 2020

#### Create a BQ dataset to host the splits

In [ ]:
client = bigquery.Client()

dataset_id = f'{PROJECT}.{BQ_DATASET_NAME}'
dataset = bigquery.Dataset(dataset_id)
dataset.location = BQ_LOCATION

try:
    dataset = client.create_dataset(dataset, timeout=30)
    print('Created dataset: ', dataset_id)
except exceptions.Conflict:
    print('Dataset {} already exists'.format(dataset_id))

#### Create a table with training features

In [ ]:
sample_size = 1000000
year = 2020

sql_script_template = '''
CREATE OR REPLACE TABLE `@PROJECT.@DATASET.@TABLE` 
AS (
    WITH
      taxitrips AS (
      SELECT
        FORMAT_DATETIME('%Y-%d-%m', trip_start_timestamp) AS date,
        trip_start_timestamp,
        trip_seconds,
        trip_miles,
        payment_type,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        tips,
        fare
      FROM
        `bigquery-public-data.chicago_taxi_trips.taxi_trips`
      WHERE 1=1 
      AND pickup_longitude IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND trip_miles > 0
      AND trip_seconds > 0
      AND fare > 0
      AND EXTRACT(YEAR FROM trip_start_timestamp) = @YEAR
    )

    SELECT
      trip_start_timestamp,
      EXTRACT(MONTH from trip_start_timestamp) as trip_month,
      EXTRACT(DAY from trip_start_timestamp) as trip_day,
      EXTRACT(DAYOFWEEK from trip_start_timestamp) as trip_day_of_week,
      EXTRACT(HOUR from trip_start_timestamp) as trip_hour,
      trip_seconds,
      trip_miles,
      payment_type,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(pickup_longitude, pickup_latitude), 0.1)
      ) AS pickup_grid,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0.1)
      ) AS dropoff_grid,
      ST_Distance(
          ST_GeogPoint(pickup_longitude, pickup_latitude), 
          ST_GeogPoint(dropoff_longitude, dropoff_latitude)
      ) AS euclidean,
      IF((tips/fare >= 0.2), 1, 0) AS tip_bin,
      CASE (ABS(MOD(FARM_FINGERPRINT(date),10))) 
          WHEN 9 THEN 'TEST'
          WHEN 8 THEN 'VALIDATE'
          ELSE 'TRAIN' END AS data_split
    FROM
      taxitrips
    LIMIT @LIMIT
)
'''

sql_script = sql_script_template.replace(
    '@PROJECT', PROJECT).replace(
    '@DATASET', BQ_DATASET_NAME).replace(
    '@TABLE', BQ_TABLE_NAME).replace(
    '@YEAR', str(year)).replace(
    '@LIMIT', str(sample_size))

job = client.query(sql_script)
job.result()

#### Review the created features

In [ ]:
sql_script = f'''
SELECT * EXCEPT (trip_start_timestamp)
FROM `{PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}`
'''
df = client.query(sql_script).result().to_dataframe()

In [ ]:
df.head().T

## Creating a tabular dataset in Vertex

### Initialize Vertex AI SDK

In [ ]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

### Create a dataset and import data

In [ ]:
display_name = 'Chicago taxi trips'
bq_source_uri = f'bq://{PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}'

filter = f'display_name="{display_name}"'

dataset = vertex_ai.TabularDataset.list(filter=filter)
if not dataset:
    print("Creating a new dataset.")
    dataset = vertex_ai.TabularDataset.create(
        display_name=display_name, bq_source=bq_source_uri,
    )

    dataset.wait()
else:
    print("Using existing dataset: ", dataset[0].resource_name)
    dataset = vertex_ai.TabularDataset(dataset_name=dataset[0].resource_name)

## Launching an AutoML training job

In [ ]:
display_name = 'Chicago Taxi classifier training'
model_display_name = 'Chicago Taxi classifier'
target_column = 'tip_bin'
optimization_prediction_type = 'classification'
optimization_objective = 'maximize-recall-at-precision'
optimization_objective_precision_value = 0.7
split_column = 'data_split'
budget_milli_node_hours = 1000

column_transformations = [
    {'categorical': {'column_name': 'trip_month'}},
    {'categorical': {'column_name': 'trip_day'}},
    {'categorical': {'column_name': 'trip_day_of_week'}},
    {'categorical': {'column_name': 'trip_hour'}},
    {'categorical': {'column_name': 'payment_type'}},
    {'categorical': {'column_name': 'pickup_grid'}},
    {'categorical': {'column_name': 'dropoff_grid'}},
    {'numeric': {'column_name': 'trip_seconds'}},
    {'numeric': {'column_name': 'euclidean'}},
    {'numeric': {'column_name': 'trip_miles'}},
]

job = vertex_ai.AutoMLTabularTrainingJob(
    display_name=display_name,
    optimization_prediction_type=optimization_prediction_type,
    optimization_objective=optimization_objective,
    optimization_objective_precision_value=optimization_objective_precision_value,
    column_transformations=column_transformations,
)

model = job.run(
    dataset=dataset,
    target_column=target_column,
    budget_milli_node_hours=budget_milli_node_hours,
    model_display_name=model_display_name,
    predefined_split_column_name=split_column,
    sync=False
)

In [ ]:
print(f"Job Name: {job.display_name}")
print(f"Job Resource Name: {job.resource_name}\n")
print(f"Check training progress at {job._dashboard_uri()}")

In [ ]:
#This blocks until the model is finished training.
model.wait()
print(f"Job Name: {model.display_name}")

## Deploy Model


In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4", sync=False)

## Model Deployment

Now deploy the trained Vertex Model resource for batch and online prediction.

For online prediction, you:

- Create an Endpoint resource for deploying the Model resource to.
- Deploy the Model resource to the Endpoint resource.
- Make online prediction requests to the Endpoint resource.

For batch-prediction, you:

- Create a batch prediction job.
- The job service will provision resources for the batch prediction request.
- The results of the batch prediction request are returned to the caller.
- The job service will unprovision the resoures for the batch prediction request.

### Predict on Endpoint - Online Prediction

In [ ]:
job.state

In [ ]:
# Block until the endpoint is deployed, which takes a few minutes.
endpoint.wait()

In [ ]:
test_instances = [  
    
    {
        "dropoff_grid": "POINT(-87.6 41.9)",
        "euclidean": 2064.2696,
        "payment_type": "Credit Card",
        "pickup_grid": "POINT(-87.6 41.9)",
        "trip_miles": 1.37,
        "trip_day": "12",
        "trip_hour": "16",
        "trip_month": "2",
        "trip_day_of_week": "4",
        "trip_seconds": "555"
    }
]

predictions = endpoint.predict(instances=test_instances)

In [ ]:
predictions

In [ ]:
predictions = endpoint.predict(instances=test_instances)
predictions

### Batch Prediction Job

Now do a batch prediction to your deployed model.

#### Make test items

In [ ]:
sql_script = f'''
SELECT trip_month, trip_day, trip_day_of_week, trip_hour, payment_type, pickup_grid, dropoff_grid, trip_seconds, euclidean, trip_miles
FROM `{PROJECT}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}`
LIMIT 1000
'''

dtypes = {
    'dropoff_grid': str,
    'euclidean': 'float64',
    'trip_month': str,
    'trip_day': str,
    'trip_day_of_week': str,
    'trip_hour': str,
    'payment_type': str,
    'pickup_grid': str,
    'trip_seconds': str,
    'trip_miles': 'float64'
}

df_test_batch = client.query(sql_script).result().to_dataframe(dtypes=dtypes)

In [ ]:
df_test_batch.head()

In [ ]:
df_test_batch.dtypes

In [ ]:
out_file_name = "bq_export_features_test.csv"
gcs_batch_request_csv = f'{STAGING_BUCKET}/test/batch/{out_file_name}'
df_test_batch.to_csv(f'{STAGING_BUCKET}/test/batch/bq_export_features_test.csv',
                     header=True, 
                     index=False,
                     quoting=csv.QUOTE_NONNUMERIC,
                     escapechar="\\",
                     doublequote=False
                    )

In [ ]:
!gsutil cat $gcs_batch_request_csv  | head

#### Make the batch prediction request

Now that your Model resource is trained, you can make a batch prediction by invoking the batch_request() method, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `gcs_source`: A list of one or more batch request input files.
- `gcs_destination_prefix`: The Cloud Storage location for storing the batch prediction resuls.
- `sync`: If set to True, the call will block while waiting for the asynchronous batch job to complete.

In [ ]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name=f"{model_display_name}-batch-{TIMESTAMP}",
    gcs_source=gcs_batch_request_csv,
    instances_format="csv",
    gcs_destination_prefix=f'{STAGING_BUCKET}/test/batch_results/',
    predictions_format="csv",
    sync=False
)

print(batch_predict_job)

#### Wait for completion of batch prediction job
Next, wait for the batch job to complete.

In [ ]:
batch_predict_job.wait()

#### Get the predictions
Next, get the results from the completed batch prediction job.

The results are written to the Cloud Storage output bucket you specified in the batch prediction request. You call the method iter_outputs() to get a list of each Cloud Storage file generated with the results. Each file contains one or more prediction requests in a JSON format:

- `content`: The prediction request.
- `prediction`: The prediction response.
    - `ids`: The internal assigned unique identifiers for each prediction request.
    - `displayNames`: The class names for each class label.
    - `confidences`: The predicted confidence, between 0 and 1, per class label.

---

**NOTE: There is issue with batch prediction job where input data types are not matching with model inputs. Skip the section below if you hit into issues**

---

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

In [ ]:
tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

## Clean up

### Undeploy Models
When you are done doing predictions, you undeploy the Model resource from the Endpoint resouce. This deprovisions all compute resources and ends billing for the deployed model.

In [ ]:
endpoint.list_models()

In [ ]:
endpoint.undeploy_all()

### Delete Endpoint

In [ ]:
endpoint.delete()

### Delete Model

In [ ]:
model.delete()